In [1]:
pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 56.6 MB/s eta 0:00:00


In [4]:
from gurobipy import Model, GRB

In [24]:
# Create a new model
model = Model("equestrian_schedule")

# Define your problem data
weeks = list(range(1, 13))  # Weeks numbered from 1 to 12
horses = list(range(1, 6))  # 5 horses, numbered from 1 to 5
events = list(range(1, 20))  # Events numbered from 1 to 12

# Define prize money for each event
prize_money = {
    1: 32923.90,
    2: 32923.90,
    3: 14308.05,
    4: 41847.20,
    5: 14308.05,
    6: 41847.20,
    7: 14308.05,
    8: 79694.30,
    9: 28154.55,
    10: 14616.75,
    11: 83386.70,
    12: 41539.50,
    13: 15231.15,
    14: 83386.70,
    15: 46770.40,
    16: 14616.75,
    17: 44001.10,
    18: 15538.85,
    19: 105387.25
}

# Map events to weeks
event_to_week = {
    1: 1,   # Event 1 in Week 1
    2: 2,   # Event 2 in Week 2
    3: 3,   # Event 3 in Week 3
    4: 3,   # Event 4 in Week 3
    5: 4,   # Event 5 in Week 4
    6: 4,   # Event 6 in Week 4
    7: 5,   # Event 7 in Week 5
    8: 5,   # Event 8 in Week 5
    9: 6,   # Event 9 in Week 6
    10: 7,  # Event 10 in Week 7
    11: 7,  # Event 11 in Week 7
    12: 8,  # Event 12 in Week 8
    13: 9,  # Event 13 in Week 9
    14: 9,  # Event 14 in Week 9
    15: 10, # Event 15 in Week 10
    16: 11, # Event 16 in Week 11
    17: 11, # Event 17 in Week 11
    18: 12, # Event 18 in Week 12
    19: 12  # Event 19 in Week 12
}


In [25]:
# Decision variables: x[i][j] = 1 if horse j participates in event i
x = {}
for i in events:
    for j in horses:
        x[i, j] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}")

In [26]:
# Update model to integrate new variables
model.update()

In [27]:
# Objective function: Maximize prize money
# Assuming prize_money is a dictionary with prize money per week
model.setObjective(sum(prize_money[i] * x[i, j] for i in events for j in horses), GRB.MAXIMIZE)

In [29]:
# Constraint: Each horse can participate up to 6 events
for j in horses:
    model.addConstr(sum(x[i, j] for i in events) <= 6, f"MaxWeeks_Horse_{j}")

# Constraint: Athlete must participate in every event with at least one horse
for i in events:
    model.addConstr(sum(x[i, j] for j in horses) >= 1, f"MinParticipation_Week_{i}")

In [34]:
# Constraint: No more than 3 horses showing in the same week
for w in weeks:
    model.addConstr(sum(x[e, j] for e in events if event_to_week[e] == w for j in horses) <= 3, f"MaxHorses_Week_{w}")

# Add constraint: Horses cannot enter more than 2 weeks in a row
for j in horses:
    for i in range(1, len(weeks) - 1):  # Iterate over weeks, stopping two weeks before the end
        if i+2 in weeks:  # Check if the next two weeks are in the schedule
            model.addConstr(sum(x[e, j] for e in events if event_to_week[e] in [i, i+1, i+2]) <= 2, f"MaxConsecutive_Weeks_{i}_Horse_{j}")

# Add constraint: One horse cannot be in two events in the same week
for j in horses:
    for w in weeks:
        # Ensure the sum of participation in all events of a given week is at most 1 for each horse
        model.addConstr(sum(x[e, j] for e in events if event_to_week[e] == w) <= 1, f"MaxOneEventPerWeek_Horse_{j}_Week_{w}")


In [35]:
# Optimize model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 208 rows, 95 columns and 955 nonzeros
Model fingerprint: 0xf14e58b7
Variable types: 0 continuous, 95 integer (95 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+04, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint MaxOneEventPerWeek_Horse_4_Week_7 by 1.000000000

Found heuristic solution: objective 1070491.3000
Presolve removed 87 rows and 0 columns
Presolve time: 0.00s
Presolved: 121 rows, 95 columns, 595 nonzeros
Variable types: 0 continuous, 95 integer (95 binary)
Found heuristic solution: objective 1398344.6500

Root relaxation: objective 1.420

In [36]:
# Output the results by horse
for j in horses:
    print(f"Horse {j} participates in:")
    for i in events:
        if x[i, j].X > 0.5:  # If the decision variable is set to 1
            week = event_to_week[i]  # Get the week number for the event
            print(f"  - Week {week}, Event {i}")
    print("")  # Add an empty line for better readability between horses


Horse 1 participates in:
  - Week 3, Event 3
  - Week 5, Event 8
  - Week 7, Event 10
  - Week 9, Event 13
  - Week 11, Event 16
  - Week 12, Event 18

Horse 2 participates in:
  - Week 4, Event 6
  - Week 5, Event 8
  - Week 7, Event 11
  - Week 8, Event 12
  - Week 10, Event 15
  - Week 11, Event 17

Horse 3 participates in:
  - Week 3, Event 4
  - Week 4, Event 6
  - Week 7, Event 11
  - Week 9, Event 14
  - Week 10, Event 15
  - Week 12, Event 19

Horse 4 participates in:
  - Week 2, Event 2
  - Week 3, Event 4
  - Week 6, Event 9
  - Week 8, Event 12
  - Week 9, Event 14
  - Week 12, Event 19

Horse 5 participates in:
  - Week 1, Event 1
  - Week 4, Event 5
  - Week 5, Event 7
  - Week 8, Event 12
  - Week 10, Event 15
  - Week 11, Event 17



In [37]:
# Output the results
for i in events:
    for j in horses:
        if x[i, j].X > 0.5:  # If the decision variable is set to 1
            week = event_to_week[i]  # Get the week number for the event
            print(f"Week {week}, Event {i}, Horse {j}: Participate")

Week 1, Event 1, Horse 5: Participate
Week 2, Event 2, Horse 4: Participate
Week 3, Event 3, Horse 1: Participate
Week 3, Event 4, Horse 3: Participate
Week 3, Event 4, Horse 4: Participate
Week 4, Event 5, Horse 5: Participate
Week 4, Event 6, Horse 2: Participate
Week 4, Event 6, Horse 3: Participate
Week 5, Event 7, Horse 5: Participate
Week 5, Event 8, Horse 1: Participate
Week 5, Event 8, Horse 2: Participate
Week 6, Event 9, Horse 4: Participate
Week 7, Event 10, Horse 1: Participate
Week 7, Event 11, Horse 2: Participate
Week 7, Event 11, Horse 3: Participate
Week 8, Event 12, Horse 2: Participate
Week 8, Event 12, Horse 4: Participate
Week 8, Event 12, Horse 5: Participate
Week 9, Event 13, Horse 1: Participate
Week 9, Event 14, Horse 3: Participate
Week 9, Event 14, Horse 4: Participate
Week 10, Event 15, Horse 2: Participate
Week 10, Event 15, Horse 3: Participate
Week 10, Event 15, Horse 5: Participate
Week 11, Event 16, Horse 1: Participate
Week 11, Event 17, Horse 2: Parti